<a href="https://colab.research.google.com/github/Fermu25/Cursos/blob/main/Calibracion_sensores_presion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
#definir funcion
class SensorCalibrator:
    def __init__(self, offset=0):
        self.offset = offset
        self.a = 0.000761  # escala inicial
        self.b = 0         # asumiendo paso por el origen al inicio

    def ajustar_con_dos_puntos(self, bits1, presion1, bits2, presion2):
        """
        Ajusta la recta (P = a*bits + b) usando dos puntos proporcionados
        """
        x1, y1 = bits1, presion1
        x2, y2 = bits2, presion2

        self.a = (y2 - y1) / (x2 - x1)
        self.b = y1 - self.a * x1

        print(f"\n📐 Nueva ecuación de ajuste: P = {self.a:.8f} * bits + {self.b:.2f}")

    def estimar_presion(self, bits):
        """
        Estima la presión a partir de una lectura de bits
        """
        bits_corr = abs(bits - self.offset)
        return self.a * bits_corr + self.b

    def evaluar_error(self, bits, presion_real):
        """
        Calcula el error porcentual de una lectura
        """
        estimada = self.estimar_presion(bits)
        error_abs = presion_real - estimada
        error_pct = 100 * error_abs / presion_real
        print(f"🔍 Presión real: {presion_real:.2f} Pa | Estimada: {estimada:.2f} Pa")
        print(f"📊 Error: {error_abs:.2f} Pa ({error_pct:.2f}%)")
        return estimada, error_abs, error_pct


# Interfaz================================

# Inicializamos calibrador
calibrador = SensorCalibrator(offset=0)

# Paso 1: Proporcionar dos puntos de calibración
# Ejemplo: (lectura de bits, presión real en Pa)
punto_1 = (1300000, 980.7)    # ~10 cm de columna de agua
punto_2 = (2600000, 2010.0)   # ~20.5 cm de columna de agua

# Ajustar ecuación a esos dos puntos
calibrador.ajustar_con_dos_puntos(*punto_1, *punto_2)

# Paso 2: Evaluar un nuevo valor (simular lectura)
# Puedes cambiar esto por cualquier nuevo punto
nueva_lectura_bits = 1950000
presion_real_nueva = 1500.0  # valor esperado

# Evaluar error
calibrador.evaluar_error(nueva_lectura_bits, presion_real_nueva)



📐 Nueva ecuación de ajuste: P = 0.00079177 * bits + -48.60
🔍 Presión real: 1500.00 Pa | Estimada: 1495.35 Pa
📊 Error: 4.65 Pa (0.31%)


(1495.3500000000001, 4.649999999999864, 0.3099999999999909)

In [ ]:
import numpy as np

class SensorCalibrator:
    def __init__(self, offset=0):#offset en mv
        self.offset = offset
        self.a =  0.00139027  # ecuación del ajuste, pendiente inicial (Pa/bit)
        self.b = -10578.1         # ordenada al origen
        self.punto_base = (0, 0)  # primer base (se puede variar)

    def ajustar_con_nuevo_punto(self, bits2, presion2):

       # Reajusta la ecuación P = a*bits + b usando el punto base + nuevo punto

        x1, y1 = self.punto_base
        x2, y2 = bits2, presion2

        self.a = (y2 - y1) / (x2 - x1)
        self.b = y1 - self.a * x1

        print(f"\n Nueva ecuación de ajuste:")
        print(f"   P = {self.a:.8f} * bits + {self.b:.4f}")

    def estimar_presion(self, bits):
        bits_corr = abs(bits - self.offset)
        return self.a * bits_corr + self.b

    def evaluar_error(self, bits, presion_real):
        estimada = self.estimar_presion(bits)
        error_abs = presion_real - estimada
        error_pct = 100 * error_abs / presion_real
        print(f" Presión medida:     {presion_real:.2f} Pa")
        print(f"🔧 Presión estimada: {estimada:.2f} Pa")
        print(f"📊 Error: {error_abs:.2f} Pa ({error_pct:.2f}%)\n")
        return estimada, error_abs, error_pct


# ===============================
calibrador = SensorCalibrator(offset=0)

# Punto de referencia original
bits_ref = 7922484
presion_ref = 6030.93  #según ajuste original

calibrador.punto_base = (bits_ref, presion_ref)
print(f" Ecuación inicial: P = 0.000761 * bits\n")

while True:
    try:
        # Solicitar nuevo punto experimental
        bits = int(input(" Ingresa lectura de bits del sensor: "))
        presion = float(input(" Ingresa presión real correspondiente (en Pa): "))

        # Reajustar modelo
        calibrador.ajustar_con_nuevo_punto(bits, presion)

        # Evaluar el nuevo punto
        calibrador.evaluar_error(bits, presion)

        print("Puedes seguir ingresando más puntos para refinar.\n---")
    except KeyboardInterrupt:
        print("\nProceso interrumpido.")
        break
    except Exception as e:
        print(f"Error: {e}\nIntenta nuevamente.")


 Ecuación inicial: P = 0.000761 * bits


Proceso interrumpido.


In [ ]:
import numpy as np

class SensorModel:
    def __init__(self, a=0.0016112, b=-15335.8, offset_bits=-793555):
        self.a = a          # Pendiente inicial
        self.b = b          # Ordenada al origen
        self.offset = offset_bits
        self.historial = [] # Lista de (bits, presión real)

    def corregir_bits(self, raw_bits):
        return abs(raw_bits - self.offset)

    def estimar_presion(self, bits):
        bits_corr = self.corregir_bits(bits)
        return self.a * bits_corr + self.b

    def evaluar(self, bits, presion_real):
        estimada = self.estimar_presion(bits)
        error_abs = presion_real - estimada
        error_pct = 100 * error_abs / presion_real
        print(f"Presión real:     {presion_real:.2f} Pa")
        print(f"Presión estimada: {estimada:.2f} Pa")
        print(f"Error:            {error_abs:.2f} Pa ({error_pct:.2f}%)\n")
        return estimada, error_abs, error_pct

    def agregar_dato(self, bits, presion_real):
        self.historial.append((self.corregir_bits(bits), presion_real))
        if len(self.historial) >= 2:
            self.recalcular_modelo()

    def recalcular_modelo(self):
        X, Y = zip(*self.historial)
        X = np.array(X)
        Y = np.array(Y)
        A = np.vstack([X, np.ones_like(X)]).T
        self.a, self.b = np.linalg.lstsq(A, Y, rcond=None)[0]
        print(f"\n📐 Nueva ecuación ajustada:")
        print(f"   P = {self.a:.8f} * bits + {self.b:.4f}\n")


# ========================
#Uso
# ========================
modelo = SensorModel()

print("Modelo inicial: P = 0.0016112 * bits - 15335.8")
print("Introduce tus puntos de medición reales para refinar el modelo.\n")

while True:
    try:
        bits = int(input("Bits corregidos (desde Arduino): "))
        presion_real = float(input("Presión real medida (Pa): "))

        modelo.agregar_dato(bits, presion_real)
        modelo.evaluar(bits, presion_real)

    except KeyboardInterrupt:
        print("\n Proceso interrumpido.")
        break
    except Exception as e:
        print(f"Error: {e}\nIntenta nuevamente.\n")


Modelo inicial: P = 0.0016112 * bits - 15335.8
Introduce tus puntos de medición reales para refinar el modelo.

Bits corregidos (desde Arduino): 3950000
Presión real medida (Pa): 3598.94
Presión real:     3598.94 Pa
Presión estimada: -7692.98 Pa
Error:            11291.92 Pa (313.76%)


 Proceso interrumpido.


In [ ]:
import numpy as np

class SensorModel:
    def __init__(self, offset_bits=0):
        # Modelo inicial ajustado
        self.a = 0.00139027
        self.b = -10578.1
        self.err_a = 1.365e-6  # Incertidumbre de la pendiente
        self.err_b = 15.44     # Incertidumbre del intercepto
        self.offset = offset_bits
        self.historial = []

    def corregir_bits(self, raw_bits):
        return abs(raw_bits - self.offset)

    def estimar_presion(self, bits):
        bits_corr = self.corregir_bits(bits)
        presion = self.a * bits_corr + self.b
        return presion, bits_corr

    def estimar_incertidumbre(self, bits_corr):
        # Propagación de error: ΔP = sqrt[(x*Δa)^2 + (Δb)^2]
        return np.sqrt((bits_corr * self.err_a) ** 2 + self.err_b ** 2)

    def evaluar(self, bits, presion_real):
        estimada, bits_corr = self.estimar_presion(bits)
        incertidumbre = self.estimar_incertidumbre(bits_corr)
        error_abs = presion_real - estimada
        error_pct = 100 * error_abs / presion_real if presion_real != 0 else float('inf')

        print(f"\n📊 Resultado de evaluación:")
        print(f"Bits corregidos:   {bits_corr}")
        print(f"Presión real:      {presion_real:.2f} Pa")
        print(f"Presión estimada:  {estimada:.2f} Pa ± {incertidumbre:.2f} Pa")
        print(f"Error:             {error_abs:.2f} Pa ({error_pct:.2f}%)\n")
        return estimada, incertidumbre

    def agregar_dato(self, bits, presion_real):
        self.historial.append((self.corregir_bits(bits), presion_real))
        if len(self.historial) >= 2:
            self.recalcular_modelo()

    def recalcular_modelo(self):
        X, Y = zip(*self.historial)
        X = np.array(X)
        Y = np.array(Y)
        A = np.vstack([X, np.ones_like(X)]).T
        resultado = np.linalg.lstsq(A, Y, rcond=None)
        coef, residuals, _, _ = resultado

        self.a, self.b = coef

        # Cálculo de errores estándar (simple, sin validación cruzada)
        y_est = A @ coef
        resid = Y - y_est
        sse = np.sum(resid ** 2)
        dof = len(X) - 2
        var_a = sse / (dof * np.sum((X - np.mean(X)) ** 2))
        var_b = var_a * np.mean(X ** 2)
        self.err_a = np.sqrt(var_a)
        self.err_b = np.sqrt(var_b)

        print(f"\n📐 Nueva ecuación ajustada:")
        print(f"   P = {self.a:.8f} * bits + {self.b:.4f}")
        print(f"   ±{self.err_a:.2e} (pendiente), ±{self.err_b:.2f} (intercepto)\n")


# ========================
# Inicio interactivo
# ========================
print("=== Calibración de sensor de presión con incertidumbre ===\n")

try:
    offset_bits = int(input("➡️  Ingresa los bits del offset (sin presión): "))
    voltaje_offset = float(input("➡️  Ingresa el voltaje del offset (en mV): "))
    print(f"\n🔧 Offset registrado: {offset_bits} bits, {voltaje_offset} mV\n")
except Exception as e:
    print(f"Error al ingresar datos del offset: {e}")
    exit()

modelo = SensorModel(offset_bits=offset_bits)

print("Introduce tus puntos de medición reales para refinar el modelo.")
print("Presiona Ctrl+C para salir.\n")

while True:
    try:
        bits = int(input("🧪 Bits leídos desde Arduino: "))
        presion_real = float(input("🌡️  Presión real medida (Pa): "))

        modelo.agregar_dato(bits, presion_real)
        modelo.evaluar(bits, presion_real)

    except KeyboardInterrupt:
        print("\n👋 Proceso interrumpido por el usuario.")
        break
    except Exception as e:
        print(f"❌ Error: {e}\nIntenta nuevamente.\n")


=== Calibración de sensor de presión con incertidumbre ===

➡️  Ingresa los bits del offset (sin presión): 8000000
➡️  Ingresa el voltaje del offset (en mV): 38.2

🔧 Offset registrado: 8000000 bits, 38.2 mV

Introduce tus puntos de medición reales para refinar el modelo.
Presiona Ctrl+C para salir.

🧪 Bits leídos desde Arduino: 8500000
🌡️  Presión real medida (Pa): 2000

📊 Resultado de evaluación:
Bits corregidos:   500000
Presión real:      2000.00 Pa
Presión estimada:  -9882.97 Pa ± 15.46 Pa
Error:             11882.97 Pa (594.15%)


👋 Proceso interrumpido por el usuario.
